In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import json
import threading



if __name__ == "__main__":
    # Create SparkContext and StreamingContext (batch interval 5s)
    sc = SparkContext("local[2]", "FinancialDataStream")
    ssc = StreamingContext(sc, 5)

    # Connect to the TCP server
    lines = ssc.socketTextStream("127.0.0.1", 9999)
    # lines are DStreams
    # Print each RDD received (for debugging)
    lines.pprint()

    # Parse JSON lines
    transactions = lines.map(lambda line: json.loads(line))

    # Example: count number of transactions per transaction_type
    counts = transactions.map(lambda tx: (tx["transaction_type"], 1)) \
                         .reduceByKey(lambda a, b: a + b)

    counts.pprint()

    amounts = transactions.map(lambda tx: (tx["transaction_type"], tx["total_amount"])) \
                          .reduceByKey(lambda a, b: a + b)
    amounts.pprint()

    
    # Start streaming
    ssc.start()
    ssc.awaitTermination(3000)




f:\anaconda3\envs\aws\lib\site-packages\pyspark\streaming\context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


-------------------------------------------
Time: 2025-09-04 22:32:51
-------------------------------------------
{"transaction_id": "d594c691-10cd-4143-8ed7-ed54700da239", "timestamp": "2025-09-04T22:32:50.164513", "account_id": "ACC_53563", "transaction_type": "BUY", "symbol": "ZOOM", "quantity": 527, "price": 155.88, "total_amount": 82148.76, "currency": "CAD", "exchange": "NASDAQ", "customer_segment": "VIP"}

-------------------------------------------
Time: 2025-09-04 22:32:51
-------------------------------------------
('BUY', 1)

-------------------------------------------
Time: 2025-09-04 22:32:51
-------------------------------------------
('BUY', 82148.76)

-------------------------------------------
Time: 2025-09-04 22:32:54
-------------------------------------------
{"transaction_id": "17bee0c8-61ad-488d-a2c5-b6017bf25f9d", "timestamp": "2025-09-04T22:32:51.175477", "account_id": "ACC_80513", "transaction_type": "SELL", "symbol": "AMD", "quantity": 16, "price": 90.73, "tot

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "f:\anaconda3\envs\aws\lib\site-packages\py4j\clientserver.py", line 535, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
RuntimeError: reentrant call inside <_io.BufferedReader name=1860>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "f:\anaconda3\envs\aws\lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "f:\anaconda3\envs\aws\lib\site-packages\py4j\clientserver.py", line 566, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "f:\anaconda3\envs\aws\lib\site-packages\py4j\clientserver.py", line 535, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "f:\anaconda3\envs\aws\

-------------------------------------------
Time: 2025-09-04 22:33:36
-------------------------------------------

Streaming context stopped.


Py4JError: An error occurred while calling o27.awaitTermination

In [ ]:
# Global reference to ssc so the signal handler can access it
import json
import signal
import sys
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

ssc = None

def signal_handler(sig, frame):
    global ssc
    print(f"\n🔴 Caught signal {sig}, stopping streaming context gracefully...")
    if ssc:
        ssc.stop(stopSparkContext=True, stopGraceFully=True)
    sys.exit(0)

if __name__ == "__main__":
    # Register signal handlers for Ctrl+C (SIGINT) and kill (SIGTERM)
    signal.signal(signal.SIGINT, signal_handler)
    signal.signal(signal.SIGTERM, signal_handler)

    # Create SparkContext and StreamingContext (batch interval 5s)
    sc = SparkContext("local[2]", "FinancialDataStream")
    ssc = StreamingContext(sc, 5)

    # Connect to the TCP server
    lines = ssc.socketTextStream("127.0.0.1", 9999)
    lines.pprint()

    # Parse JSON lines
    transactions = lines.map(lambda line: json.loads(line))

    # Count transactions per type
    counts = transactions.map(lambda tx: (tx["transaction_type"], 1)) \
                         .reduceByKey(lambda a, b: a + b)
    counts.pprint()

    # Sum amounts per transaction type
    amounts = transactions.map(lambda tx: (tx["transaction_type"], tx["total_amount"])) \
                          .reduceByKey(lambda a, b: a + b)
    amounts.pprint()

    # Start streaming
    ssc.start()
    print("✅ Streaming started. Press Ctrl+C to stop.")
    ssc.awaitTermination(2000)
    sc.stop()
    ssc.stop()


NameError: name 'signal' is not defined

In [ ]:
import json
import threading
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

# Globals
ssc = None
sc = None
streaming_thread = None

def start_streaming():
    global sc, ssc, streaming_thread

    if sc is not None or ssc is not None:
        print("⚠️ SparkContext or StreamingContext already exists. Stop it before starting again.")
        return

    # Create SparkContext and StreamingContext (batch interval 5s)
    sc = SparkContext("local[2]", "FinancialDataStream")
    ssc = StreamingContext(sc, 5)

    # Connect to the TCP server
    lines = ssc.socketTextStream("127.0.0.1", 9999)
    lines.pprint()

    # Parse JSON lines
    transactions = lines.map(lambda line: json.loads(line))

    # Count transactions per type
    counts = transactions.map(lambda tx: (tx["transaction_type"], 1)) \
                         .reduceByKey(lambda a, b: a + b)
    counts.pprint()

    # Sum amounts per transaction type
    amounts = transactions.map(lambda tx: (tx["transaction_type"], tx["total_amount"])) \
                          .reduceByKey(lambda a, b: a + b)
    amounts.pprint()

    # Start streaming
    ssc.start()
    print("✅ Streaming started in background. Run stop_streaming() to stop it.")

    # Run awaitTermination in a separate thread so Jupyter stays free
    def run_stream():
        try:
            ssc.awaitTermination()
        except Exception as e:
            print(f"Background thread terminated: {e}")

    streaming_thread = threading.Thread(target=run_stream, daemon=True)
    streaming_thread.start()


def stop_streaming():
    global sc, ssc, streaming_thread
    if ssc:
        print("🛑 Stopping StreamingContext...")
        ssc.stop(stopSparkContext=True, stopGraceFully=True)
        ssc = None
    if sc:
        sc.stop()
        sc = None
        print("✅ SparkContext stopped.")
    if streaming_thread:
        streaming_thread.join(timeout=5)
        streaming_thread = None
        print("🟢 Streaming thread stopped.")




In [22]:
start_streaming()

✅ Streaming started in background. Run stop_streaming() to stop it.


In [23]:
stop_streaming()

🛑 Stopping StreamingContext...
✅ SparkContext stopped.


In [21]:
sc.stop()
ssc.stop()

AttributeError: 'NoneType' object has no attribute 'stop'